## Anomaly Detection Project

Project Scenario

Hello,

I have some questions for you that I need to be answered before the board meeting Friday afternoon. I need to be able to speak to the following questions. I also need a single slide that I can incorporate into my existing presentation (Google Slides) that summarizes the most important points. My questions are listed below; however, if you discover anything else important that I didn’t think to ask, please include that as well.


1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?
5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?
6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?
7. Which lessons are least accessed?
8. Anything else I should be aware of?


Thank you,

---

### Executive Summary

**Goal**

To detect patterns and anomalous activity in a dataset that monitors the Codeup curriculum access. 

**Key Findings:**

* The prepared dataframe has over 847,000 rows and 16 columns.

* There are 4 different programs available in the dataset:
    * 1. PHP Full Stack (Web Development)
    * 2. Java Full Stack (Web Development)
    * 3. Data Science
    * 4. Front End (Web Development)
    
* There seems to be some suspicious activity going on with one user accessing 272 pages in the curriculum over a  short duration.
    * This user also used a few different IP addresses.
    
* There were 468 pages in the curriculum which were only accessed once.
    * Some of these pages were found to be lessons while the others were professional development pages.
    
* The following lessons were accessed post graduation:
    * Data Science: SQL, Classification Overview, Classification Scaling
    * Full-Stack Java: Javascript-I, HTML-CSS, Spring
    * Front End: Javascript-I, HTML-CSS, Spring


In [1]:
# import libraries and custom modules to use 

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import math
from sklearn import metrics
from random import randint
from matplotlib import style
import seaborn as sns
from wrangle import *
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler

In [2]:
# acquire data

df = get_logs_df()

In [3]:
# look at data

df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2.0


In [4]:
# look at the information for our dataframe

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 900223 entries, 0 to 900222
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        900223 non-null  object 
 1   time        900223 non-null  object 
 2   path        900222 non-null  object 
 3   user_id     900223 non-null  int64  
 4   cohort_id   847330 non-null  float64
 5   ip          900223 non-null  object 
 6   id          847330 non-null  float64
 7   name        847330 non-null  object 
 8   slack       847330 non-null  object 
 9   start_date  847330 non-null  object 
 10  end_date    847330 non-null  object 
 11  created_at  847330 non-null  object 
 12  updated_at  847330 non-null  object 
 13  deleted_at  0 non-null       float64
 14  program_id  847330 non-null  float64
dtypes: float64(4), int64(1), object(10)
memory usage: 109.9+ MB


In [5]:
# call prepare function

df = prep_logs(df)

In [6]:
# look at prepared df

df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 847329 entries, 2018-01-26 09:55:03 to 2021-04-21 16:44:39
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date            847329 non-null  datetime64[ns]
 1   time            847329 non-null  datetime64[ns]
 2   path            847329 non-null  object        
 3   user_id         847329 non-null  int64         
 4   cohort_id       847329 non-null  float64       
 5   ip              847329 non-null  object        
 6   id              847329 non-null  float64       
 7   name            847329 non-null  object        
 8   slack           847329 non-null  object        
 9   start_date      847329 non-null  datetime64[ns]
 10  end_date        847329 non-null  datetime64[ns]
 11  created_at      847329 non-null  datetime64[ns]
 12  updated_at      847329 non-null  datetime64[ns]
 13  program_id      847329 non-null  float64       
 14  ac

In [7]:
# look at prepared df

df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,program_id,accessed_after,program_name
2018-01-26 09:55:03,2018-01-26,2021-10-21 09:55:03,/,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,1,Web Dev - Front End
2018-01-26 09:56:02,2018-01-26,2021-10-21 09:56:02,java-ii,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,1,Web Dev - Front End
2018-01-26 09:56:05,2018-01-26,2021-10-21 09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,1,Web Dev - Front End
2018-01-26 09:56:06,2018-01-26,2021-10-21 09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,1,Web Dev - Front End
2018-01-26 09:56:24,2018-01-26,2021-10-21 09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2.0,0,Web Dev - Java


### 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?

There are 4 different programs available in the dataset:
1. PHP Full Stack (Web Development)
2. Java Full Stack (Web Development)
3. Data Science
4. Front End (Web Development)

Checked the value counts for each program's paths to see which lesson was referred to the most across cohorts. The following results were found:

- The most commonly accessed page was the home page for all cohorts
- For the Web Development Java programs, the javascript-i lesson was accessed the most.
- For Data Science, the classification overview lesson was accessed the most.
- For the Web Development Front-End Program, the javascript-i lesson was accessed the most.

In [9]:
# check different programs offered and proportion of students in each

df.program_id.value_counts(dropna=False,normalize=True)

2.0    0.841898
3.0    0.122044
1.0    0.036052
4.0    0.000006
Name: program_id, dtype: float64

In [10]:
# find the lesson that is most accessed for each program

df_program_list = list(df.program_name.value_counts().index)

#return the top, most visited path for each program

for program in df_program_list: 
    
        #remove paths that are the home page, table of contents, and other miscellaneous
        
        program_df = pd.DataFrame(df[(df.path != '/')&(df.path != 'toc')&(df.path != 'search/search_index.json')&(df.path != 'index.html')].groupby('program_name')['path'].value_counts())
        access_df = program_df.loc[[program]]
        access_df = access_df.path.nlargest(1)
        print(access_df)
        print('\n')
        print('----------------------------------------------')
        print('\n')

program_name    path        
Web Dev - Java  javascript-i    17457
Name: path, dtype: int64


----------------------------------------------


program_name  path                   
Data Science  classification/overview    1785
Name: path, dtype: int64


----------------------------------------------


program_name         path        
Web Dev - Front End  javascript-i    736
Name: path, dtype: int64


----------------------------------------------




### 2. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

Yes, there are some students who accessed the curriculum less than 50 times in the period that they were active. I used this as a threshold because that would mean they accessed the curriculum less than twice a week during their active training. 

I found that the 34 of the students belonged to the Web Development (Full Stack Java) cohorts and only 3 students were from Data Science cohorts. 

None of the students who hardly accessed the curriculum were from Front End and Full Stack PHP programs.

The hardly_working dataframe below provides more information about students who hardly accessed the program curriculum.

In [11]:
# find students who accessed curriculum while still active students

df_current = df.loc[df.accessed_after == 0]

In [12]:
# df with students who accessed curriculum while still in the program

df_current = df_current.loc[(df_current.index >= df_current.start_date) & (df_current.index <= df_current.end_date)]

In [13]:
# check the df

df_current

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,program_id,accessed_after,program_name
2018-01-26 09:56:24,2018-01-26,2021-10-21 09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2.0,0,Web Dev - Java
2018-01-26 09:56:41,2018-01-26,2021-10-21 09:56:41,javascript-i/loops,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2.0,0,Web Dev - Java
2018-01-26 09:56:46,2018-01-26,2021-10-21 09:56:46,javascript-i/conditionals,3,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2.0,0,Web Dev - Java
2018-01-26 09:56:48,2018-01-26,2021-10-21 09:56:48,javascript-i/functions,3,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2.0,0,Web Dev - Java
2018-01-26 09:56:59,2018-01-26,2021-10-21 09:56:59,javascript-i/loops,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2.0,0,Web Dev - Java
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-21 16:36:09,2021-04-21,2021-10-21 16:36:09,jquery/personal-site,869,135.0,136.50.98.51,135.0,Marco,#marco,2021-01-25,2021-07-19,2021-01-20 21:31:11,2021-01-20 21:31:11,2.0,0,Web Dev - Java
2021-04-21 16:36:34,2021-04-21,2021-10-21 16:36:34,html-css/css-ii/bootstrap-grid-system,948,138.0,104.48.214.211,138.0,Neptune,#neptune,2021-03-15,2021-09-03,2021-03-15 19:57:09,2021-03-15 19:57:09,2.0,0,Web Dev - Java
2021-04-21 16:37:48,2021-04-21,2021-10-21 16:37:48,java-iii,834,134.0,67.11.50.23,134.0,Luna,#luna,2020-12-07,2021-06-08,2020-12-07 16:58:43,2020-12-07 16:58:43,2.0,0,Web Dev - Java
2021-04-21 16:38:14,2021-04-21,2021-10-21 16:38:14,java-iii/servlets,834,134.0,67.11.50.23,134.0,Luna,#luna,2020-12-07,2021-06-08,2020-12-07 16:58:43,2020-12-07 16:58:43,2.0,0,Web Dev - Java


In [14]:
# create a df with value counts for each user id

user_df = pd.DataFrame(df_current.user_id.value_counts(dropna=False)).reset_index().\
                rename(index=str, columns={'index': 'user_id', 'user_id': 'user_count'})

# create df for each user id and its probability

user_df2 = pd.DataFrame(df_current.user_id.value_counts(dropna=False)/df_current.user_id.count()).reset_index().\
                rename(index=str, columns={'index': 'user_id', 'user_id': 'user_proba'})

# merge the 2 user dfs

user_df = user_df.merge(user_df2)

In [15]:
# merge user_df with the current students df, set date as index and fill null values

df_current = df_current.merge(user_df, on=['user_id'], how='left').fillna(value=0).set_index('date')
df_current.user_proba.value_counts()

0.004552    5892
0.007359    4763
0.002131    4137
0.006296    4075
0.002770    3586
            ... 
0.000009       6
0.000008       5
0.000002       4
0.000006       4
0.000005       3
Name: user_proba, Length: 612, dtype: int64

In [16]:
# students who access curriculum less than 50 times are considered hardly accessing it

hardly_working = df_current[df_current.user_count < 50]

In [17]:
# pull specific columns in this df

hardly_working = hardly_working[['user_id','name','program_name','start_date','end_date', 'user_count']]

hardly_working

,user_id,name,program_name,start_date,end_date,user_count
date,,,,,,
2018-01-26,24,Sequoia,Web Dev - Java,2017-09-27,2018-02-15,26
2018-01-26,24,Sequoia,Web Dev - Java,2017-09-27,2018-02-15,26
2018-01-26,41,Sequoia,Web Dev - Java,2017-09-27,2018-02-15,28
2018-01-26,49,Sequoia,Web Dev - Java,2017-09-27,2018-02-15,25
2018-01-26,49,Sequoia,Web Dev - Java,2017-09-27,2018-02-15,25
...,...,...,...,...,...,...
2021-04-21,976,Oberon,Web Dev - Java,2021-04-12,2021-10-01,29
2021-04-21,973,Oberon,Web Dev - Java,2021-04-12,2021-10-01,35
2021-04-21,972,Oberon,Web Dev - Java,2021-04-12,2021-10-01,34


In [18]:
# reset index so date is not the index

hardly_working = hardly_working.reset_index()
hardly_working

,date,user_id,name,program_name,start_date,end_date,user_count
0,2018-01-26,24,Sequoia,Web Dev - Java,2017-09-27,2018-02-15,26
1,2018-01-26,24,Sequoia,Web Dev - Java,2017-09-27,2018-02-15,26
2,2018-01-26,41,Sequoia,Web Dev - Java,2017-09-27,2018-02-15,28
3,2018-01-26,49,Sequoia,Web Dev - Java,2017-09-27,2018-02-15,25
4,2018-01-26,49,Sequoia,Web Dev - Java,2017-09-27,2018-02-15,25
...,...,...,...,...,...,...,...
800,2021-04-21,976,Oberon,Web Dev - Java,2021-04-12,2021-10-01,29
801,2021-04-21,973,Oberon,Web Dev - Java,2021-04-12,2021-10-01,35
802,2021-04-21,972,Oberon,Web Dev - Java,2021-04-12,2021-10-01,34
803,2021-04-21,981,Neptune,Web Dev - Java,2021-03-15,2021-09-03,42


In [19]:
# drop any duplicate user ids in the df and keep the most recent access

hardly_working = hardly_working.drop_duplicates(subset='user_id', keep="last")
hardly_working

,date,user_id,name,program_name,start_date,end_date,user_count
69,2018-02-05,49,Sequoia,Web Dev - Java,2017-09-27,2018-02-15,25
71,2018-02-07,24,Sequoia,Web Dev - Java,2017-09-27,2018-02-15,26
78,2018-02-08,41,Sequoia,Web Dev - Java,2017-09-27,2018-02-15,28
82,2018-09-27,278,Voyageurs,Web Dev - Java,2018-05-29,2018-10-11,4
100,2019-01-22,340,Zion,Web Dev - Java,2019-01-22,2019-06-04,18
138,2019-03-19,388,Andromeda,Web Dev - Java,2019-03-18,2019-07-30,8
150,2019-03-29,371,Yosemite,Web Dev - Java,2018-11-05,2019-04-03,42
167,2019-08-30,487,Bayes,Data Science,2019-08-19,2020-01-30,17
173,2019-11-04,539,Europa,Web Dev - Java,2019-11-04,2020-04-17,5
208,2019-11-08,529,Europa,Web Dev - Java,2019-11-04,2020-04-17,36


In [20]:
# check what different programs users belong to

hardly_working.program_name.value_counts()

Web Dev - Java    34
Data Science       3
Name: program_name, dtype: int64

In [21]:
# find different cohorts that users belong to who don't access the curriculum often

hardly_working.name.value_counts()


Oberon       12
Neptune       4
Sequoia       3
Europa        3
Marco         2
Darden        2
Ganymede      2
Jupiter       2
Voyageurs     1
Andromeda     1
Bayes         1
Zion          1
Hyperion      1
Fortuna       1
Yosemite      1
Name: name, dtype: int64

In [22]:
# check start date of cohorts

hardly_working.start_date.value_counts()

2021-04-12    12
2021-03-15     4
2019-11-04     3
2017-09-27     3
2021-01-25     2
2020-07-13     2
2020-03-23     2
2020-09-21     2
2019-01-22     1
2020-01-13     1
2018-11-05     1
2019-08-19     1
2020-05-26     1
2018-05-29     1
2019-03-18     1
Name: start_date, dtype: int64

### 3. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?

- It appears that there may be some web scraping happening.
- Explored the anomalies in number of page views beyond the upper bollinger band.
- User 341 had some suspicious activity with 272 pages viewed on 2019-03-03 and 104 pages viewed on 2020-04-21. 
- The user used a few different ip addresses on 03-03-2019 and the curriculum was accessed after 10pm to midnight.
- The user also accessed a wide range of topics. 
- On 2020-04-21, the user accessed a wide range of topics and viewed 104 pages using a different ip address.


In [23]:
# most frequently used ip

df.ip.value_counts().idxmax()

'97.105.19.58'

In [24]:
def prep(df, user):
    '''
    This function takes in a df and a single user. 
    It converts the date column to a datetime object and sets that as the index.
    It also resamples daily counts for number of page views.
    '''
    df = df[df.user_id == user]
    pages = df['path'].resample('d').count()
    return pages

def compute_pct_b(pages, span, weight, user):
    '''
    This function takes in the pages, the number of days, the weight to calculate the standard deviation and the user.
    It calculates the mid, upper and lower bollinger bands and concatenates the upper and lower bands.
    It then creates a new dataframe with the number of page view, midband, upper and lower bands.
    It calculate the %b and adds a user id column to the df and then returns the df.
    '''
    midband = pages.ewm(span=span).mean()
    stdev = pages.ewm(span=span).std()
    ub = midband + stdev * weight
    lb = midband - stdev * weight
    bb = pd.concat([ub, lb], axis=1)
    my_df = pd.concat([pages, midband, bb], axis=1)
    my_df.columns = ['pages', 'midband', 'ub', 'lb']
    my_df['pct_b'] = (my_df['pages'] - my_df['lb'])/(my_df['ub'] - my_df['lb'])
    my_df['user_id'] = user
    return my_df

def find_anomalies(df, user, span, weight):
    '''
    This function takes in a df, user id, number of days as span and the weight to compute %b. 
    It then prepares the df by calling the prep function.
    It also creates a new dataframe by calling the compute pct b function.
    It returns a subset of the df where %b is greater than 1.
    '''
    pages = prep(df, user)
    my_df = compute_pct_b(pages, span, weight, user)
    return my_df[my_df.pct_b > 1]

In [25]:
# test on a single user

user = 68
span = 30
weight = 3

# initialize empty df

anomalies = pd.DataFrame()

# call function to create a new df with anomalies for the user 

user_df = find_anomalies(df, user, span, weight)

# concatenate the above df created with the empty anomalies df

anomalies = pd.concat([anomalies, user_df], axis=0)

In [26]:
# find anomalous activity for user id 68

anomalies

,pages,midband,ub,lb,pct_b,user_id
2018-05-04,67,7.035731,60.259707,-46.188246,1.063320,68
2018-06-25,50,4.814414,45.293820,-35.664992,1.058131,68
2018-08-20,17,1.240614,15.592025,-13.110798,1.049054,68
2018-10-16,13,0.898680,10.970734,-9.173375,1.100737,68
2018-11-15,27,1.863468,22.364497,-18.637561,1.113055,68
2019-01-30,21,1.407805,17.354013,-14.538403,1.114321,68
2019-03-25,24,1.622558,19.932129,-16.687012,1.111086,68
2019-06-27,9,0.587115,7.439000,-6.264770,1.113910,68
2019-07-10,14,1.554444,13.962988,-10.854099,1.001491,68
2019-08-07,18,1.966101,17.408340,-13.476138,1.019157,68


In [27]:
# loop through all users to find anomalies for each

span = 30
weight = 3.5

anomalies = pd.DataFrame()
for u in list(df.user_id.unique()):
    user_df = find_anomalies(df, u, span, weight)
    anomalies = pd.concat([anomalies, user_df], axis=0)

In [28]:
# find anomalous activity for all users

anomalies

,pages,midband,ub,lb,pct_b,user_id
2019-01-29,44,3.623334,42.081344,-34.834677,1.024945,1
2019-07-08,101,8.248768,96.507472,-80.009936,1.025451,1
2018-10-15,9,0.581421,8.457414,-7.294571,1.034446,3
2019-01-09,4,0.262470,3.802115,-3.277175,1.027953,3
2019-04-06,2,0.129825,1.889712,-1.630061,1.031334,3
...,...,...,...,...,...,...
2021-01-24,7,0.574393,6.857226,-5.708440,1.011362,817
2021-03-25,59,6.824556,58.415342,-44.766230,1.005666,843
2021-02-23,67,8.229409,66.322904,-49.864086,1.005828,851
2021-04-06,19,1.348943,18.056879,-15.358994,1.028224,854


- A pct_b value above 1 indicates pages accessed are beyond upper bollinger band. 
- In this sitatuation, we care about students who are accessing the curriculum more often than 3 standard deviations.

In [29]:
# look at value counts for pages in anomalies without sorting the counts

anomalies.pages.value_counts(sort = False)

1      42
2      35
3      65
4      74
5      52
       ..
165     1
179     1
192     1
198     1
272     1
Name: pages, Length: 97, dtype: int64

In [30]:
# look at max value counts for pages in anomalies

anomalies.pages.max()

272

The highest number of page views is 272

In [31]:
# look at a subset of anomalies df where the number of page views is equal to 272

anomalies[anomalies.pages == 272]

,pages,midband,ub,lb,pct_b,user_id
2019-03-03,272,24.721632,266.780128,-217.336864,1.010782,341


- Looks like it is user 341 who has a very high number of page views. Will investigate this user's activity further to check if there is anything suspicous about their IP address.

In [32]:
# pull a subset of data where user id is 341

anomalies[anomalies.user_id == 341]

,pages,midband,ub,lb,pct_b,user_id
2019-03-03,272,24.721632,266.780128,-217.336864,1.010782,341
2020-02-18,3,0.193548,2.817185,-2.430088,1.034840,341
2020-04-21,109,7.250725,102.413277,-87.911826,1.034608,341
2021-02-09,10,0.659630,9.508602,-8.189342,1.027766,341


- On 2019-03-03 and 2020-04-21 this user had very high number of page views. 

In [33]:
# check different ip addresses for this user

df.ip[df.user_id == 341].value_counts()

97.105.19.58       677
172.124.70.146     462
173.174.243.231    233
204.44.112.76      180
70.123.203.131      25
192.171.117.210      4
Name: ip, dtype: int64

- This user seems to be using a few different IP addresses to access the curriculum. 
- The ip's with suspicious activity are:
    - 97.105.19.58
    - 172.124.70.146
    - 173.174.243.231    
    - 204.44.112.76     
- We will look at a subset of the df to further investigate this user's details for the dates of 03-03-2019 and 04-21-2020.

In [34]:
sus_1 = df.loc['2019-03-02':'2019-03-04']

sus_2 = df.loc['2020-04-20':'2020-04-21']

In [35]:
# set to display all rows and columns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [36]:
# check details for user 341 on 3/3

sus_1[sus_1.user_id == 341]

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,program_id,accessed_after,program_name
2019-03-03 21:14:08,2019-03-03,2021-10-21 21:14:08,/,341,29.0,173.174.243.231,29.0,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,2.0,0,Web Dev - Java
2019-03-03 21:14:14,2019-03-03,2021-10-21 21:14:14,jquery,341,29.0,173.174.243.231,29.0,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,2.0,0,Web Dev - Java
2019-03-03 21:14:27,2019-03-03,2021-10-21 21:14:27,jquery/essential-methods/traversing,341,29.0,173.174.243.231,29.0,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,2.0,0,Web Dev - Java
2019-03-03 21:15:25,2019-03-03,2021-10-21 21:15:25,jquery/effects,341,29.0,173.174.243.231,29.0,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,2.0,0,Web Dev - Java
2019-03-03 22:52:05,2019-03-03,2021-10-21 22:52:05,html-css,341,29.0,204.44.112.76,29.0,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,2.0,0,Web Dev - Java
2019-03-03 22:52:06,2019-03-03,2021-10-21 22:52:06,javascript-i,341,29.0,204.44.112.76,29.0,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,2.0,0,Web Dev - Java
2019-03-03 22:52:06,2019-03-03,2021-10-21 22:52:06,java-i,341,29.0,204.44.112.76,29.0,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,2.0,0,Web Dev - Java
2019-03-03 22:52:06,2019-03-03,2021-10-21 22:52:06,java-ii,341,29.0,204.44.112.76,29.0,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,2.0,0,Web Dev - Java
2019-03-03 22:52:06,2019-03-03,2021-10-21 22:52:06,javascript-ii,341,29.0,204.44.112.76,29.0,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,2.0,0,Web Dev - Java
2019-03-03 22:52:06,2019-03-03,2021-10-21 22:52:06,jquery,341,29.0,204.44.112.76,29.0,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,2.0,0,Web Dev - Java


In [37]:
# check details for user 341 on 4/21

sus_2[sus_2.user_id == 341]

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,program_id,accessed_after,program_name
2020-04-21 17:45:43,2020-04-21,2021-10-21 17:45:43,/,341,29.0,172.124.70.146,29.0,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,2.0,1,Web Dev - Java
2020-04-21 17:45:52,2020-04-21,2021-10-21 17:45:52,/,341,29.0,172.124.70.146,29.0,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,2.0,1,Web Dev - Java
2020-04-21 17:45:56,2020-04-21,2021-10-21 17:45:56,appendix,341,29.0,172.124.70.146,29.0,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,2.0,1,Web Dev - Java
2020-04-21 18:00:36,2020-04-21,2021-10-21 18:00:36,javascript-ii,341,29.0,172.124.70.146,29.0,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,2.0,1,Web Dev - Java
2020-04-21 18:23:01,2020-04-21,2021-10-21 18:23:01,/,341,29.0,172.124.70.146,29.0,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,2.0,1,Web Dev - Java
2020-04-21 18:24:37,2020-04-21,2021-10-21 18:24:37,appendix,341,29.0,172.124.70.146,29.0,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,2.0,1,Web Dev - Java
2020-04-21 18:24:38,2020-04-21,2021-10-21 18:24:38,appendix,341,29.0,172.124.70.146,29.0,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,2.0,1,Web Dev - Java
2020-04-21 18:26:56,2020-04-21,2021-10-21 18:26:56,/,341,29.0,172.124.70.146,29.0,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,2.0,1,Web Dev - Java
2020-04-21 18:27:54,2020-04-21,2021-10-21 18:27:54,appendix,341,29.0,172.124.70.146,29.0,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,2.0,1,Web Dev - Java
2020-04-21 18:27:56,2020-04-21,2021-10-21 18:27:56,appendix,341,29.0,172.124.70.146,29.0,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,2.0,1,Web Dev - Java


### 4. Which lessons are least accessed?

There were 467 pages in the curriculum which were only accessed once. In this situation, we are assuming that the least accessed pages would be accessed at least once. Some of the least accessed pages were found to be lessons while the others were professional development pages. There was also one lesson page that seemed erroneous. The following were some of the least accessed lessons:

- javascript-i/intruduction/operators                              
- creating-charts                                                  
- appendix/capstone-workbook/project-proposal                      
- content/examples/examples/examples/gitbook/images/favicon.ico    
- javascri                                                         
- content/examples/php/technology-companies.php                    
- relationships/indexes                                            
- extra-features/file-upload                                       
- quic/115                                                         
- interview-questions 

In [38]:
df.path.value_counts(sort=True).tail(10)

javascript-i/intruduction/operators                              1
creating-charts                                                  1
appendix/capstone-workbook/project-proposal                      1
content/examples/examples/examples/gitbook/images/favicon.ico    1
javascri                                                         1
content/examples/php/technology-companies.php                    1
relationships/indexes                                            1
extra-features/file-upload                                       1
quic/115                                                         1
interview-questions                                              1
Name: path, dtype: int64

In [39]:
# number of pages that were accessed once

df.path.value_counts(sort=True).nsmallest(1, keep='all').count()

467

In [41]:
# all the pages that were accessed once

df.path.value_counts(sort=True).nsmallest(1, keep='all')

appendix/java                                                                                                                                                                                                                                                              1
student                                                                                                                                                                                                                                                                    1
statistics-assessment                                                                                                                                                                                                                                                      1
f                                                                                                                                                                                                

### 5. What topics are grads continuing to reference after graduation and into their jobs (for each program)?

- We created the accessed_after variable that is 0 if a student did not access the lesson after graduation and 1 if they did. We found the top 3 lessons graduates of each program access were as follows.

- Java Full Stack (Web Development) 
    - javascript-i  
    - spring          
    - html-css 
    
- Data Science  
    - Mysql-overview                          
    - classification/overview                   
    - classification/scale_features_or_not
                                   
- Font End (Web Development)
    - javascript-i  
    - spring          
    - html-css 


In [8]:
# create a new df which only shows students who accessed curriculum after graduation

df_grads = df.loc[df.accessed_after==1]

df_grads

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,program_id,accessed_after,program_name
2018-01-26 09:55:03,2018-01-26,2021-10-21 09:55:03,/,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,1,Web Dev - Front End
2018-01-26 09:56:02,2018-01-26,2021-10-21 09:56:02,java-ii,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,1,Web Dev - Front End
2018-01-26 09:56:05,2018-01-26,2021-10-21 09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,1,Web Dev - Front End
2018-01-26 09:56:06,2018-01-26,2021-10-21 09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,1,Web Dev - Front End
2018-01-26 10:14:47,2018-01-26,2021-10-21 10:14:47,/,11,1.0,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,1,Web Dev - Front End
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-21 16:41:51,2021-04-21,2021-10-21 16:41:51,jquery/personal-site,64,28.0,71.150.217.33,28.0,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,2.0,1,Web Dev - Java
2021-04-21 16:42:02,2021-04-21,2021-10-21 16:42:02,jquery/mapbox-api,64,28.0,71.150.217.33,28.0,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,2.0,1,Web Dev - Java
2021-04-21 16:42:09,2021-04-21,2021-10-21 16:42:09,jquery/ajax/weather-map,64,28.0,71.150.217.33,28.0,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,2.0,1,Web Dev - Java
2021-04-21 16:44:37,2021-04-21,2021-10-21 16:44:37,anomaly-detection/discrete-probabilistic-methods,744,28.0,24.160.137.86,28.0,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,2.0,1,Web Dev - Java


In [9]:
# how many students access the curriculum after graduation

len(df_grads)

200132

In [10]:
# turn program names into a list

df_program_list = list(df.program_name.value_counts().index)

df_program_list

['Web Dev - Java', 'Data Science', 'Web Dev - Front End']

In [11]:
# for loop removes non lesson pages and prints top lessons grads access

for program in df_program_list:     
        program_df = pd.DataFrame(df_grads[(df_grads.path != '/')&(df_grads.path != 'toc')&(df_grads.path != 'search/search_index.json')&(df_grads.path != 'index.html')].groupby('program_name')['path'].value_counts())
        
        # create a separate df for each program
        
        access_df = program_df.loc[[program]]
        
        # find top 3 lessons for each program
        
        access_df = access_df.path.nlargest(3)
        print(access_df)
        print('----------------------------------------------')
        print('\n')

program_name    path        
Web Dev - Java  javascript-i    4233
                spring          3771
                html-css        3145
Name: path, dtype: int64
----------------------------------------------


program_name  path                                    
Data Science  sql/mysql-overview                          275
              classification/overview                     267
              classification/scale_features_or_not.svg    220
Name: path, dtype: int64
----------------------------------------------


program_name         path        
Web Dev - Front End  javascript-i    736
                     html-css        542
                     spring          501
Name: path, dtype: int64
----------------------------------------------




### Conclusion

- Top lessons for Data Science included SQL and Classification while those for Full Stack Java and Front End included Javascript I, Spring and HTML-CSS.
- Front End and Full Stack Java programs graduates accessed the same top 3 lessons after graduation.
- There is risk of webscraping/crawling and malicious activity as evidenced by one of the students' activity.
- Students with low access rates usually stopped utilizing the program within 3 days of their cohort start date.
- 468 pages in the curriculum were only accessed once.